In [18]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import chi2_contingency

In [75]:
data = pd.read_csv('churn_analysis.csv',header=0,index_col=0)

Признаки:

state — штат США

account_length — длительность использования аккаунта

area_code — деление пользователей на псевдорегионы, использующееся в телекоме

intl_plan — подключена ли у пользователя услуга международного общения

vmail_plan — подключена ли у пользователя услуга голосовых сообщений

vmail_message — количество голосых сообщений, который пользователь отправил / принял

day_calls — сколько пользователь совершил дневных звонков

day_mins — сколько пользователь проговорил минут в течение дня

day_charge — сколько пользователь заплатил за свою дневную активность

eve_calls, eve_mins, eve_charge — аналогичные метрики относительно вечерней активности

night_calls, night_mins, night_charge — аналогичные метрики относительно ночной активности

intl_calls, intl_mins, intl_charge — аналогичные метрики относительно международного общения

custserv_calls — сколько раз пользователь позвонил в службу поддержки

treatment — номер стратегии, которая применялись для удержания абонентов (0, 2 = два разных типа воздействия, 1 = контрольная группа)

mes_estim — оценка интенсивности пользования интернет мессенджерами

churn — результат оттока: перестал ли абонент пользоваться услугами оператора


In [76]:
data.head()

,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,eve_mins,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,KS,128,415,no,yes,25,265.1,110,45.07,197.4,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
1,OH,107,415,no,yes,26,161.6,123,27.47,195.5,...,254.4,103,11.45,13.7,3,3.70,1,0,0.55,False.
2,NJ,137,415,no,no,0,243.4,114,41.38,121.2,...,162.6,104,7.32,12.2,5,3.29,0,0,0.72,False.
3,OH,84,408,yes,no,0,299.4,71,50.90,61.9,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
4,OK,75,415,yes,no,0,166.7,113,28.34,148.3,...,186.9,121,8.41,10.1,3,2.73,3,2,0.45,False.


In [77]:
data['churn'] = data['churn'].map({'False.': 0, 'True.': 1})

In [78]:
# splitting data into groups of loyal clients and churn
data_churn = data[data['churn'] == 1]
data_loyal = data[data['churn'] == 0]

In [85]:
# processing for forming contingency table
print 'No zeros among loyal clients? {}'.format(data_loyal[data_loyal['treatment'] == 1].groupby('state').size().size ==\
                                    data.groupby('state').size().size)
print 'No zeros among churn clients? {}'.format(data_churn[data_churn['treatment'] == 1].groupby('state').size().size ==\
                                    data.groupby('state').size().size)

b = data_loyal[data_loyal['treatment'] == 1].groupby('state').size()
a = data.groupby('state').size()
for i in range(data.groupby('state').size().size):
    a.iloc[i] = 0
for i in data_churn[data_churn['treatment'] == 1].groupby('state').size().index:
    a.loc[i] = data_churn[data_churn['treatment'] == 1].groupby('state').size().loc[i]
    


No zeros among loyal clients? True
No zeros among churn clients? False


In [91]:
p_values = []
for i,first_state in enumerate(b.index):
    for j,second_state in enumerate(a.index):
        if i < j:
            a11 = b.loc[first_state]
            a12 = a.loc[first_state]
            a21 = b.loc[second_state]
            a22 = a.loc[second_state]
            table = np.array([a11,a12,a21,a22]).reshape([2,2])
            p_values.append(chi2_contingency(table,correction=False)[1])

In [92]:
alpha = 0.05
ans1 = sum([1 if x<alpha else 0 for x in p_values])

print "The number of pairs for which p-value < alpha : %d" %ans1


The number of pairs for which p-value < alpha : 34


chi2 test for all the states at once:

In [96]:
chi2_contingency(np.array([b.values,a.values]).T,correction=False)[1]

0.70975900427784733